# Introdução
Esse arquivo foi feito para usar as analises em conjunto de forma estruturada para encontrar as melhores combinações de números baseado na frequencia e no intervalo de desaparecimento dos mesmos.

Para isso foi utilizado as sequintes analises:
* Frequência total de cada número
* Todos os intervalos de desaparecimento de cada número

Ainda estou aperfeiçoando a analise com o objetivo de melhorar a precisão das combinações posseiveis, estou estudando mais estatisca, machine learning e analise de dados para obter exito.

Pretendo criar outras analises para complementar e estrutura ainda mais as analises para ser mais robusto a tomada de decisão.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import bar_chart_race as bcr
import matplotlib as mpl
import os
from openpyxl import load_workbook
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4

caminho = r"c:\Users\castr\OneDrive\Área de Trabalho\Repositorios\scripts\Analise-loterias\data\processed"

os.chdir(caminho)

mega = pd.read_excel('Dados_Update.xlsx')

table_for_analysis = mega.drop(['Concurso', 'Data'], axis=1)


## Frequência total e individual de cada bola

Código abaixo gera uma tabela com as frequencias de cada número por bola e frequencia total 

In [12]:
contagem = {}

for numero in range(1, 61):
    contagem_numero = {}
    for coluna in table_for_analysis.columns:
        temp = 0
        for instancia in table_for_analysis[coluna]:
            if instancia == numero:
                temp += 1
        
        contagem_numero[coluna] = temp
    
    contagem[numero] = contagem_numero

tabela = pd.DataFrame(contagem).T

tabela.index.name = 'Número'

tabela['Total'] = tabela.sum(axis=1)

caminho_original = os.getcwd()

os.chdir(r"c:\Users\castr\OneDrive\Área de Trabalho\Repositorios\scripts\Analise-loterias\reports\Metricas\MegaSena")

tabela.to_excel("tabela_frequencias.xlsx", sheet_name="Frequencia")

os.chdir(caminho_original)

display(tabela)

,bola 1,bola 2,bola 3,bola 4,bola 5,bola 6,Total
Número,,,,,,,
1,42,47,55,56,39,44,283
2,56,46,56,43,47,46,294
3,40,55,40,34,48,52,269
4,59,44,55,51,48,49,306
5,52,67,42,51,52,55,319
6,53,53,46,41,48,50,291
7,53,43,41,48,45,41,271
8,41,59,45,44,43,55,287
9,43,45,43,45,54,43,273


## Tempo de desaparecimento

O código abaixo gera todos os intervalos de cada número onde cada um ficou desaparecido

In [13]:
numeros = {i: [] for i in range(1, 61)}

contadores = {i: 0 for i in range(1, 61)}

for num in range(1, 61):
    for index, row in table_for_analysis.iterrows():

        if num in row.values:

            numeros[num].append(contadores[num])
            
            contadores[num] = 0
        else:
            
            contadores[num] += 1

tabela_Total_Desaparecimento = pd.DataFrame.from_dict(numeros, orient='index')

os.chdir(r"c:\Users\castr\OneDrive\Área de Trabalho\Repositorios\scripts\Analise-loterias\reports\Metricas\MegaSena")

tabela_Total_Desaparecimento.to_excel('Intervalos de Desaparecimento.xlsx', sheet_name='Intervalos Históricos')

display(tabela_Total_Desaparecimento)


,0,1,2,3,4,5,6,7,8,9,...,330,331,332,333,334,335,336,337,338,339
1,13,13,16,15,3,2,2,3,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,2,14,16,3,2,0,0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,21,10,20,8,9,3,34,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,4,30,0,1,8,8,10,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,16,9,6,2,0,5,2,0,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,13,21,4,5,11,0,14,9,5,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,51,14,5,14,4,24,9,2,8,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,6,5,15,7,12,0,15,7,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,14,0,6,11,1,6,31,3,32,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,11,5,7,15,0,11,10,8,9,7,...,11.0,2.0,34.0,6.0,2.0,42.0,11.0,6.0,1.0,13.0


## Complemento do tempo de desaparecimento

O código abaixo retorna o maior e menor intervalo de tempo que um número ficou desaparecido, como bónus ele também dá a media de desaparecimento de cada número 

In [14]:
lista = {}
maximo = {}

for i in range(1,61):
    valores = numeros.get(i)
    
    lista[i] = round(sum(valores) / len(valores), 2)

    maximo[i] = max(valores)

tabela_Media_Desaparecimento = pd.DataFrame.from_dict(lista, orient='index', columns=['Tempo de Desaparecimento Médio'])

tabela_Desaparecimento_maximo = pd.DataFrame.from_dict(maximo, orient='index', columns=['Tempo de Desaparecimento Máximo'])

os.chdir(r"c:\Users\castr\OneDrive\Área de Trabalho\Repositorios\scripts\Analise-loterias\reports\Metricas\MegaSena")

with pd.ExcelWriter("Intervalos de Desaparecimento.xlsx", engine='openpyxl',mode='a', if_sheet_exists='replace') as writer:
    tabela_Media_Desaparecimento.to_excel(writer, sheet_name='Média Desaparecimento', index=True, )
    tabela_Desaparecimento_maximo.to_excel(writer, sheet_name='Máximo Desaparecimento', index=True)

display(tabela_Desaparecimento_maximo)
display(tabela_Media_Desaparecimento)

,Tempo de Desaparecimento Máximo
1,49
2,61
3,51
4,55
5,46
6,58
7,60
8,60
9,53
10,54


,Tempo de Desaparecimento Médio
1,9.27
2,8.88
3,9.80
4,8.51
5,8.12
6,8.99
7,9.72
8,9.11
9,9.66
10,7.55
